In [1]:
%%html

<style>
img[alt="1"] {
    max-width: 600px;
}
</style>

# RNN

![1](img/7.png)

![1](img/4.png)

![1](img/5.png)

![1](img/6.png)

## from vanilla RNN to ...

![1](img/8.png)

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

/home/seohasong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def tprint(tensor):
    
    try:
        print(tensor.name)
        print(sess.run(tensor), "\n")
    except:
        print(sess.run(tensor), "\n")

# SIMPLE RNN by tensorflow

![1](img/9.png)
![1](img/10.png)

In [4]:
char_rdic = ['h', 'e', 'l', 'o']
char_dic = {w : i for i, w in enumerate(char_rdic)}
print ("\nchar_dic\n", char_dic)
ground_truth = [char_dic[c] for c in 'hello']
print ("\nground_thruth\n", ground_truth)
rnn_size = len(char_dic) # 4
rnn_size = 4
batch_size = 1
output_size = 4


char_dic
 {'h': 0, 'e': 1, 'l': 2, 'o': 3}

ground_thruth
 [0, 1, 2, 2, 3]


In [251]:
g = tf.Graph()
with g.as_default():
    
    sess = tf.Session()

In [252]:
with g.as_default():
    
    targets = tf.one_hot(ground_truth[1:], len(char_dic), 1.0, 0.0, -1, name="targets")
    tprint(targets)
    
    x_data = tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1, name="x_data")
    tprint(x_data)

targets:0
[[ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]] 

x_data:0
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]] 



In [253]:
with g.as_default():
    
    x_split = tf.split(x_data, len(char_dic), 0)
    for tensor in x_split:
        tprint(tensor)

    initial_state = tf.zeros([batch_size, rnn_size], name="initial_state")
    tprint(initial_state)

split:0
[[ 1.  0.  0.  0.]] 

split:1
[[ 0.  1.  0.  0.]] 

split:2
[[ 0.  0.  1.  0.]] 

split:3
[[ 0.  0.  1.  0.]] 

initial_state:0
[[ 0.  0.  0.  0.]] 



In [254]:
with g.as_default():

    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=rnn_size)
    outputs, state = tf.nn.static_rnn(cell=rnn_cell, inputs=x_split, initial_state=initial_state)
    sess.run(tf.global_variables_initializer())
    tprint(state)
    
    logits = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size], name="logits")
    tprint(logits)

rnn/rnn/basic_rnn_cell/Tanh_3:0
[[ 0.35424188  0.33717629  0.64027423 -0.39660347]] 

logits:0
[[ 0.37931871  0.33856413  0.45600885 -0.3346543 ]
 [-0.36572421 -0.72284687  0.32072175  0.26666909]
 [-0.24722229  0.77427912  0.4912532  -0.25607264]
 [ 0.35424188  0.33717629  0.64027423 -0.39660347]] 



In [255]:
with g.as_default():

    loss = tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=logits)
    tprint(loss)
    
    cost = tf.reduce_mean(loss, name="cost")
    tprint(cost)
    
    max_index = tf.argmax(logits, 1, name="max_index")
    tprint(max_index)
    
    train_op = tf.train.AdamOptimizer(0.01, 0.9).minimize(cost)

Reshape_2:0
[ 1.30190146  1.03066885  1.18742037  2.08220339] 

cost:0
1.40055 

max_index:0
[2 2 1 2] 



In [256]:
with g.as_default():
    
    sess.run(tf.global_variables_initializer())

    
for i in range(100):

    sess.run(train_op)
    result = sess.run(max_index)
    
    if (i) % 10 == 9:
        print("cost: %s" % sess.run(cost))
        print(result, [char_rdic[t] for t in result])

cost: 1.05317
[1 1 2 2] ['e', 'e', 'l', 'l']
cost: 0.886339
[1 2 2 2] ['e', 'l', 'l', 'l']
cost: 0.792356
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.72703
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.668132
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.612854
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.566362
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.535386
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.513138
[1 2 2 3] ['e', 'l', 'l', 'o']
cost: 0.497599
[1 2 2 3] ['e', 'l', 'l', 'o']


# LSTM

## Vanishing Gradient Problem

![1](img/11.png)

## cell state, forget gate, input gate, output gate

![1](img/15.png)
![1](img/12.png)

## cell state

![1](img/13.png)

## forget gate

![1](img/14.png)

## input gate

![1](img/16.png)

## 문맥을 파악하는 과정

![1](img/17.png)

## output gate

![1](img/18.png)

# RNN and LSTM by tensorflow

![1](img/19.png)
![1](img/20.png)

In [17]:
pd.read_csv("RNN_Data/train.csv", encoding='cp949')
# pd.read_csv("RNN_Data/train.csv")

Unnamed: 0  review_point                                        review_text
0     5591273             1         실망이 매우큽니다...기대하고 봤는데 아니이게뭔가- -;;전혀 내용없네요  
1     8456128             1                                 1점도 아깝다 영화로 고뮨하냐  
2     7874891             1                                          시간뺏겼다아오  
3     1283306             1  항상 뻔한 반일 소설 레퍼토리. 언제까지 근거없는 반일 영화로 돈벌래? ㅋㅋㅋ 그리...
4     5655629             1                                   한마디로 shit 입니다.  
5     3570284            10                             체드 마이클 머레이.. 넘멋져요 ㅋㅋ  
6      261834            10                                       좋은영화입니다bbb  
7      130568            10                              감 동 실화라서 더 더욱 감동입니다  
8     2683925            10      2편에서도 이어지는 어정쩡한 로맨스는 여전히 아쉽고 할리 베리는 여전히 예쁘다  
9     1726836            10  영화 정말정말 재미있었지만 소설의 내용은 거의 들어가지 않았다... 광인이 너무 리...

In [18]:
def Train_generator(path):
    
    train = pd.read_csv(path, encoding='cp949')
#     train = pd.read_csv(path)
    
    y = np.array([[1, 0] if score == 1 else [0, 1] for score in train['review_point']])
    
    listed_sentences = [review.split() for review in train['review_text']]
    max_sentence_length = max(len(sentence) for sentence in listed_sentences)
    wordset = list(set(sum(listed_sentences, [])))
    
    X = [[[0 for _ in range(len(wordset))]
          for _ in range(max_sentence_length)]
         for _ in range(len(listed_sentences))]
    
    for i, sentence in enumerate(listed_sentences):
        for i_, word in enumerate(sentence):
            X[i][i_][wordset.index(word)] = 1

    return np.array(X), y

In [19]:
def feed_batch(batch_idx):
    
    xs, ys = train_X[batch_idx], train_y[batch_idx]
    
    return {X: xs, y: ys}

In [21]:
train_X, train_y = Train_generator("RNN_Data/train.csv")
print(train_X)
print(train_y)

[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 1 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 ..., 
 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [22]:
rnn_size = train_X.shape[1]
print("\nrnn_size\n", rnn_size)
input_size = train_X.shape[2]
print("\ninput_size\n", input_size)
batch_size = 6


rnn_size
 24

input_size
 75


In [23]:
batch_idx = np.random.choice(train_X.shape[0], batch_size, False)
g = tf.Graph()
with g.as_default():
    
    sess = tf.Session()
    

In [24]:
with g.as_default():
    
    X = tf.placeholder(tf.float32, shape=[None, rnn_size, input_size], name="X")
    y = tf.placeholder(tf.float32, shape=[None, 2], name="y")
    print("X and y\n", sess.run([X, y], feed_batch(batch_idx)), "\n")

    initial_state = tf.zeros([batch_size, 10], name="initial_state")
    tprint(initial_state)

X and y
 [array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.

In [25]:
with g.as_default():    
    
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
#     lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=5, state_is_tuple=False)
    
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, X, initial_state=initial_state)
    sess.run(tf.global_variables_initializer())
    print(sess.run(state, feed_batch(batch_idx)))

[[ -8.98344399e-10   4.70149750e-08   5.41128742e-08  -8.64249259e-08
   -1.37436743e-07  -6.09535391e-08   5.25861985e-08   8.50197690e-09
    1.95127157e-07   1.46544053e-07]
 [ -2.47571632e-01  -5.08854128e-02  -1.49647474e-01   1.96838990e-01
    7.65587538e-02   2.31271446e-01  -1.99216902e-01   1.43690079e-01
    3.08823213e-02  -1.02944799e-01]
 [ -4.32593019e-08  -3.98685494e-11  -1.69635701e-08   1.72521375e-08
   -5.57919932e-08  -5.33788054e-08   1.01224691e-08   1.01412132e-08
    5.95735337e-08   2.75949930e-08]
 [ -5.20633439e-05  -2.83408706e-04  -3.17498576e-04   1.02749225e-04
   -9.68932218e-05  -5.63432113e-04  -1.02033409e-04  -2.18764297e-04
   -5.40138979e-04  -6.35060132e-04]
 [  1.52296291e-08  -5.51660628e-09  -5.99659167e-09  -1.65972291e-09
    3.79008682e-08   1.52339918e-08  -1.18515808e-09  -1.36222003e-08
   -4.55460771e-08  -3.13303730e-08]
 [ -7.43646569e-06  -2.68697681e-06  -6.20657693e-06   7.57873613e-06
   -1.94629388e-06  -6.33422724e-06  -8.20552

In [26]:
with g.as_default():
    
    weight = tf.truncated_normal([10, 2], mean=0, stddev=1)
    print("\nweight\n", sess.run(weight))
    
    bias = tf.Variable(tf.constant(0.1, shape=[2]))
    sess.run(tf.global_variables_initializer())
    print("\nbias\n", sess.run(bias))

    fc = tf.matmul(state, weight) + bias
    print("\nfc\n", sess.run(fc, feed_batch(batch_idx)))    
    
    probability = tf.nn.softmax(fc)
    print("\nprobability\n", sess.run(probability, feed_batch(batch_idx))) 


weight
 [[ 1.17741036  0.80181885]
 [ 0.07659792  0.94633675]
 [ 0.40152454 -0.14563446]
 [-0.63956797  0.59423876]
 [ 1.03757811  0.32858944]
 [-1.48490357 -1.41161835]
 [-0.54542899 -0.5629611 ]
 [ 0.58980119  1.44688833]
 [ 1.19874299 -1.01361883]
 [ 0.16240348 -0.2258918 ]]

bias
 [ 0.1  0.1]

fc
 [[ 0.10000762  0.09999368]
 [-0.00828488  0.38134813]
 [ 0.10000405  0.09999663]
 [ 0.09887543  0.1018512 ]
 [ 0.10000397  0.0999967 ]
 [ 0.10003208  0.09997334]]

probability
 [[ 0.49999881  0.50000125]
 [ 0.5068267   0.49317327]
 [ 0.49999934  0.50000066]
 [ 0.50011832  0.4998816 ]
 [ 0.49999937  0.50000066]
 [ 0.49999484  0.50000513]]


In [27]:
with g.as_default():
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=fc))
    print("\ncost\n", sess.run(cost, feed_batch(batch_idx)))

    predict = tf.argmax(fc, 1)
    print("\npredict\n", sess.run(predict, feed_batch(batch_idx)))
    
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, tf.argmax(y, 1)), dtype=tf.float32))
    print("\naccuracy\n", sess.run(accuracy, feed_batch(batch_idx)))
    
    train_op = tf.train.AdamOptimizer(1e-3, 0.3).minimize(cost)


cost
 0.720162

predict
 [1 1 1 0 1 1]

accuracy
 0.5


In [30]:
with g.as_default():
    
    sess.run(tf.global_variables_initializer())

    
for i in range(20001):
    
    batch_idx = np.random.choice(train_X.shape[0], batch_size, False)
    _, p, a, c = sess.run([train_op, predict, accuracy, cost], feed_batch(batch_idx))
    
    if i % 4000 == 0:
        print("step : %d, train accuracy : %g, train loss : %g"%(i, a, c))
        print(p)
#         print(pd.read_csv("RNN_Data/train.csv", encoding='cp949').iloc[batch_idx])
        print(pd.read_csv("RNN_Data/train.csv", encoding='cp949').iloc[batch_idx])        

step : 0, train accuracy : 0.166667, train loss : 0.712115
[1 0 0 1 0 1]
   Unnamed: 0  review_point                                        review_text
1     8456128             1                                 1점도 아깝다 영화로 고뮨하냐  
9     1726836            10  영화 정말정말 재미있었지만 소설의 내용은 거의 들어가지 않았다... 광인이 너무 리...
7      130568            10                              감 동 실화라서 더 더욱 감동입니다  
3     1283306             1  항상 뻔한 반일 소설 레퍼토리. 언제까지 근거없는 반일 영화로 돈벌래? ㅋㅋㅋ 그리...
8     2683925            10      2편에서도 이어지는 어정쩡한 로맨스는 여전히 아쉽고 할리 베리는 여전히 예쁘다  
5     3570284            10                             체드 마이클 머레이.. 넘멋져요 ㅋㅋ  
step : 4000, train accuracy : 0.333333, train loss : 0.682017
[0 0 0 0 0 0]
   Unnamed: 0  review_point                                        review_text
2     7874891             1                                          시간뺏겼다아오  
5     3570284            10                             체드 마이클 머레이.. 넘멋져요 ㅋㅋ  
9     1726836            10  영화 정말정말 재미있었지만 소설의 내용은 거의 들어가지 않

# DLSTM, LSTMP


![1](img/1.png)

![1](img/2.png)